# JRA Scraping Notebook

中央競馬（JRA）のデータをスクレイピングし、Google Drive上のデータセットに追加します。

In [ ]:
# 1. Google Driveのマウント
from google.colab import drive
drive.mount('/content/drive')

import os
import sys

# ★★★ 設定項目 ★★★
# scraperフォルダが存在するパス (Google Drive上のパス)
# 例: '/content/drive/MyDrive/dai-keiba'
PROJECT_PATH = '/content/drive/MyDrive/dai-keiba'

if not os.path.exists(PROJECT_PATH):
    print(f"Error: Path {PROJECT_PATH} does not exist. Please check your Drive structure.")
else:
    print(f"Project path found: {PROJECT_PATH}")
    os.chdir(PROJECT_PATH)
    sys.path.append(PROJECT_PATH)


In [ ]:
# 2. 必要なライブラリのインポート
try:
    import pandas as pd
    import requests
    import bs4
except ImportError:
    !pip install pandas requests beautifulsoup4
    import pandas as pd
    import requests
    import bs4

from datetime import datetime, date
from scraper.jra_scraper import scrape_jra_year, JRA_MONTH_PARAMS
import time

# ==========================================
# 🚨 2023年/2022年パラメータ不足の修正パッチ
# ==========================================
# パラメータマップを上書き更新
try:
    JRA_MONTH_PARAMS.update({
        "2023": { "01": "27", "02": "F5", "03": "C3", "04": "91", "05": "5F", "06": "2D", "07": "FB", "08": "C9", "09": "97", "10": "06", "11": "D4", "12": "A2" },
        "2022": { "01": "9B", "02": "69", "03": "37", "04": "05", "05": "D3", "06": "A1", "07": "6F", "08": "3D", "09": "0B", "10": "7A", "11": "48", "12": "16" }
    })
    print("✅ JRA 2023/2022 Parameters have been patched successfully.")
except NameError:
    print("⚠️ JRA_MONTH_PARAMS not found in scope. Ensure imports are correct.")


In [ ]:
# 2.1 スクレイパーの修正パッチ (コース詳細・天候取得)
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import scraper.jra_scraper

def patched_scrape_jra_race(url, existing_race_ids=None):
    print(f'Accessing JRA URL (Patched): {url}...')
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.encoding = 'cp932'
        if response.status_code != 200: return None
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # (Skipping full redundant parse code for brevity in this patch, assuming main logic matches jra_scraper.py)
        # Ideally we should just RELOAD the module if we updated the file in Drive.
        # If the user updates the file in Drive (scraper/jra_scraper.py), they just need to reload.
        # But to be safe, we can force reload.
        pass
    except: pass
    return scraper.jra_scraper.scrape_jra_race(url, existing_race_ids)

import importlib
importlib.reload(scraper.jra_scraper)
from scraper.jra_scraper import scrape_jra_race, scrape_jra_year, JRA_MONTH_PARAMS
print('✅ Loaded updated scraper/jra_scraper.py from Drive')


In [ ]:
# 3. スクレイピング実行関数の定義

def jra_scrape_execution(year_str, start_date=None, end_date=None):
    CSV_FILE_PATH = os.path.join(PROJECT_PATH, "database.csv")
    print(f"Using CSV Path: {CSV_FILE_PATH}")

    def save_chunk(df_chunk):
        if os.path.exists(CSV_FILE_PATH):
            try:
                # Read types as string to prevent auto-float for IDs
                existing_df = pd.read_csv(CSV_FILE_PATH, dtype={'race_id': str, 'horse_id': str})
                combined_df = pd.concat([existing_df, df_chunk], ignore_index=True)
            except Exception as e:
                print(f"Read Error: {e}, creating new.")
                combined_df = df_chunk
        else:
            combined_df = df_chunk

        # Deduplicate
        subset_cols = ['race_id', '馬名']
        subset_cols = [c for c in subset_cols if c in combined_df.columns]
        if subset_cols:
            combined_df.drop_duplicates(subset=subset_cols, keep='last', inplace=True)

        combined_df.to_csv(CSV_FILE_PATH, index=False, encoding="utf-8-sig")
        print(f"  [Saved] Total rows: {len(combined_df)} (+{len(df_chunk)} new)")

    print(f"Starting Scraping for {year_str} ({start_date} ~ {end_date})")

    # Load existing IDs to skip
    existing_ids = set()
    if os.path.exists(CSV_FILE_PATH):
        try:
             df_e = pd.read_csv(CSV_FILE_PATH, usecols=['race_id'], dtype={'race_id': str})
             existing_ids = set(df_e['race_id'].astype(str))
             print(f"  Loaded {len(existing_ids)} existing race IDs to skip.")
        except:
             pass

    scrape_jra_year(year_str, start_date=start_date, end_date=end_date, save_callback=save_chunk, existing_race_ids=existing_ids)


In [ ]:
# 4. 実行パラメータの設定と開始
# -----------------------------
TARGET_YEAR = "2024"
TARGET_MONTH = 1  # ★何月を取得するか指定 (Noneの場合は全期間、1〜12を指定)

import calendar
from datetime import date

START_DATE = None
END_DATE = None

if TARGET_MONTH:
    # 指定した月の1日〜末日を設定
    _, last_day = calendar.monthrange(int(TARGET_YEAR), int(TARGET_MONTH))
    START_DATE = date(int(TARGET_YEAR), int(TARGET_MONTH), 1)
    END_DATE = date(int(TARGET_YEAR), int(TARGET_MONTH), last_day)
    print(f"Targeting specific month: {START_DATE} to {END_DATE}")
else:
    # 自動判定ロジック (既存データの翌日から)
    CSV_FILE_PATH = os.path.join(PROJECT_PATH, "database.csv")
    if os.path.exists(CSV_FILE_PATH):
        try:
             df_exist = pd.read_csv(CSV_FILE_PATH)
             if '日付' in df_exist.columns and not df_exist.empty:
                 df_exist['date_obj'] = pd.to_datetime(df_exist['日付'], format='%Y年%m月%d日', errors='coerce')
                 last_date = df_exist['date_obj'].max()
                 if pd.notna(last_date):
                     # START_DATE = last_date.date() # 旧: 続きから
                     # 新: 欠落補完のために、強制的にその年の1月1日からスキャンする (existing_idsでスキップされる)
                     START_DATE = date(int(TARGET_YEAR), 1, 1)
                     print(f"既存データの最終日時: {last_date.date()} (欠落確認のため {START_DATE} からスキャンします)")
        except Exception as e:
            print(f"既存データ確認エラー: {e}")

print(f"Scraping Target: {TARGET_YEAR}, Start: {START_DATE}, End: {END_DATE}")
jra_scrape_execution(TARGET_YEAR, start_date=START_DATE, end_date=END_DATE)


In [ ]:
# 4.1 データベースカラム名の修正 (重複改善)
import os
import pandas as pd

def fix_database_columns_notebook():
    # Google Driveパスを使用
    target_file = os.path.join(PROJECT_PATH, 'database.csv')
    
    if not os.path.exists(target_file):
        print('Database file not found.')
        return

    print(f'Checking columns in {target_file}...')
    try:
        df = pd.read_csv(target_file, low_memory=False)
        
        # 表記ゆれ辞書
        columns_to_fix = {
            'コーナー通過順': 'コーナー 通過順',
            '馬体重(増減)': '馬体重 (増減)'
        }
        
        changed = False
        for bad_col, good_col in columns_to_fix.items():
            if bad_col in df.columns:
                print(f'Merging {bad_col} -> {good_col}...')
                if good_col not in df.columns:
                    df[good_col] = df[bad_col]
                else:
                    df[good_col] = df[good_col].fillna(df[bad_col])
                
                df.drop(columns=[bad_col], inplace=True)
                changed = True
        
        if changed:
            df.to_csv(target_file, index=False, encoding='utf-8-sig')
            print('✅ Database columns fixed and saved.')
        else:
            print('No column fixes needed.')
            
    except Exception as e:
        print(f'Error fixing columns: {e}')

fix_database_columns_notebook()


In [ ]:
# 4.2 欠損データの補完 (HorseID & 過去走)
import pandas as pd
import sys
import os
import re
from datetime import datetime
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading

# Ensure scraper path
sys.path.append(os.path.join(PROJECT_PATH, 'scraper'))
from scraper.race_scraper import RaceScraper

# Global lock for dataframe update
df_lock = threading.Lock()

def fetch_race_horse_ids(rid):
    scraper = RaceScraper()
    try:
        url = f'https://race.netkeiba.com/race/result.html?race_id={rid}'
        soup = scraper._get_soup(url)
        if not soup: return None
        
        table = soup.find('table', id='All_Result_Table')
        if not table: return None
        
        horse_map = {}
        rows = table.find_all('tr', class_='HorseList')
        for row in rows:
            name_tag = row.select_one('.Horse_Name a')
            if name_tag:
                h_name = name_tag.text.strip()
                href = name_tag.get('href', '')
                match = re.search(r'/horse/(\d+)', href)
                if match:
                    horse_map[h_name] = match.group(1)
        return (rid, horse_map)
    except Exception as e:
        print(f'Error fetching race {rid}: {e}')
        return None

def fetch_horse_history(horse_id):
    scraper = RaceScraper()
    try:
        df = scraper.get_past_races(str(horse_id), n_samples=None)
        return (horse_id, df)
    except Exception as e:
        print(f'Error fetching horse {horse_id}: {e}')
        return (horse_id, pd.DataFrame())

def fill_missing_past_data_notebook():
    csv_path = os.path.join(PROJECT_PATH, 'database.csv')
    if not os.path.exists(csv_path):
        print(f'Error: {csv_path} not found.')
        return

    print(f'Reading {csv_path}...')
    df = pd.read_csv(csv_path)

    if '日付' in df.columns:
        df['date_dt'] = pd.to_datetime(df['日付'], format='%Y年%m月%d日', errors='coerce')
    else:
        print('Error: 日付 column not found.')
        return

    if 'horse_id' not in df.columns:
        df['horse_id'] = None

    # 1. Fill Missing Horse IDs
    if 'race_id' in df.columns:
        missing_mask = df['horse_id'].isna() | (df['horse_id'] == '')
        if missing_mask.any():
            races_to_update = df.loc[missing_mask, 'race_id'].unique()
            print(f'Need to fetch IDs for {len(races_to_update)} races...')
            
            with ThreadPoolExecutor(max_workers=5) as executor:
                futures = {executor.submit(fetch_race_horse_ids, rid): rid for rid in races_to_update}
                completed = 0
                for future in as_completed(futures):
                    completed += 1
                    if completed % 10 == 0: print(f'  [IDs] {completed}/{len(races_to_update)}')
                    result = future.result()
                    if result:
                        rid, horse_map = result
                        if horse_map:
                            indices = df[df['race_id'] == rid].index
                            for idx in indices:
                                h_name = df.at[idx, '馬名']
                                if h_name in horse_map:
                                    df.at[idx, 'horse_id'] = horse_map[h_name]
            
            df.to_csv(csv_path, index=False, encoding='utf-8-sig')
            print('Saved updated IDs.')
        else:
            print('All Horse IDs present.')

    # 2. Fill Past History
    fields_map = {
        'date': 'date', 'rank': 'rank', 'time': 'time', 'run_style': 'run_style',
        'race_name': 'race_name', 'last_3f': 'last_3f', 'horse_weight': 'horse_weight',
        'jockey': 'jockey', 'condition': 'condition', 'odds': 'odds',
        'weather': 'weather', 'distance': 'distance', 'course_type': 'course_type'
    }

    unique_horses = df['horse_id'].dropna().unique()
    # Simplification: Only fetch for horses missing past data to save time?
    # For now, let's keep full logic but maybe we should check if columns exist
    # If columns don't exist, create them
    for k in fields_map.keys():
        for i in range(1, 6):
            col = f'past_{i}_{k}'
            if col not in df.columns:
                df[col] = None

    print(f'Found {len(unique_horses)} unique horses. Fetching history...')
    history_store = {}

    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = {executor.submit(fetch_horse_history, hid): hid for hid in unique_horses}
        completed = 0
        for future in as_completed(futures):
            completed += 1
            if completed % 50 == 0: print(f'  [History] {completed}/{len(unique_horses)}')
            hid, hist_df = future.result()
            history_store[hid] = hist_df

    print('Applying history data...')
    for idx, row in df.iterrows():
        hid = row.get('horse_id')
        current_date = row.get('date_dt')
        if pd.notna(hid) and hid in history_store:
            hist_df = history_store[hid]
            if hist_df.empty: continue
            
            if 'date' in hist_df.columns:
                # Fix Date format in hist_df if needed (YYYY/MM/DD)
                hist_df['date_obj'] = pd.to_datetime(hist_df['date'], errors='coerce')
            
            if 'date_obj' not in hist_df.columns: continue
            if pd.isna(current_date): continue
            
            past_races = hist_df[hist_df['date_obj'] < current_date].sort_values('date_obj', ascending=False).head(5)
            
            for i, (p_idx, p_row) in enumerate(past_races.iterrows()):
                n = i + 1
                if n > 5: break
                for k, v in fields_map.items():
                    df.at[idx, f'past_{n}_{k}'] = p_row.get(v)

    if 'date_dt' in df.columns: df.drop(columns=['date_dt'], inplace=True)
    df.to_csv(csv_path, index=False, encoding='utf-8-sig')
    print('Done filling past data.')

fill_missing_past_data_notebook()


In [ ]:
# 5. データ加工 (Feature Engineering) の実行
# ----------------------------------------
# スクレイピングした database.csv から学習用データを生成します
from ml.feature_engineering import calculate_features

INPUT_CSV = os.path.join(PROJECT_PATH, "database.csv")
OUTPUT_CSV = os.path.join(PROJECT_PATH, "processed_data.csv")

if os.path.exists(INPUT_CSV):
    print("Starting Feature Engineering...")
    calculate_features(INPUT_CSV, OUTPUT_CSV)
    print("Done!")
else:
    print("Error: database.csv not found.")
